# Tossing Dominos
_Following http://images.math.cnrs.fr/Pavages-aleatoires-par-touillage?lang=fr_

In [ ]:
from sage.graphs.generators.families import AztecDiamondGraph
class OrderedDomino:
    def __init__(self, first, second):
        self.first = first
        self.second = second
        if first[0] == second[0]:
            self.direction = 'horizontal'
        else:
            self.direction = 'vertical'
    def parity(self, i):
        return (self.first[0] % 2 + self.first[1] % 2 + i) % 2
    def __repr__(self):
        return "OrderedDomino from %s to %s" % (self.first, self.second)

In [ ]:
def apply_matching(m, size, n):
    matching = {}
    shift_val = size - n
    def shift(t):
        return (t[0] + shift_val, t[1] + shift_val)
    for first, second in m:
        if first[0] < second[0] or first[1] < second[1]:
            d = OrderedDomino(shift(first), shift(second))
        else:
            d = OrderedDomino(shift(second), shift(first))
        matching[shift(first)] = d
        matching[shift(second)] = d
    return matching
def figure(size):
    g = AztecDiamondGraph(size)
    m = (((0,0),(0,1)), ((1,0), (1,1)))
    return g, m, apply_matching(m, size, 1)
def parity(pos):
    return (pos[0]%2 + pos[1]%2)%2
def similar_position(pos, ref):
    return ((pos[0]%2 + pos[1]%2)%2 == (ref[0]%2 + ref[1]%2)%2)

In [ ]:
def tossing(g, size, n, matching):
    verts = AztecDiamondGraph(n).vertices()
    shift_val = size - n
    def shift(t):
        return (t[0] + shift_val, t[1] + shift_val)
    shifted_verts = [shift(v) for v in verts]
    ref_corner = shift((0, min([v[1] for v in verts if v[0]==0])))
    new_matching = {}
    for v in shifted_verts:
        # is it an "active cell"?
        if not similar_position(v, ref_corner):
            continue
        bottom_left = (v[0]+1, v[1])
        if not bottom_left in shifted_verts:
            continue
        top_right = (v[0], v[1]+1)
        if not top_right in shifted_verts:
            continue
        bottom_right = (v[0]+1, v[1]+1)
        if not bottom_right in shifted_verts:
            continue
        if v in matching.keys() and matching[v].first == v and \
        ((matching[v].direction == 'horizontal' and bottom_left in matching.keys() \
          and matching[bottom_left].first == bottom_left and matching[bottom_left].second == bottom_right) \
            or (matching[v].direction == 'vertical' and top_right in matching.keys() \
                and matching[top_right].first == top_right and matching[top_right].second == bottom_right)):
            # Ignore both dominos
            pass
        elif v in matching.keys() and matching[v].first == v:
            # Slide
            if matching[v].direction == 'horizontal':
                new_matching[bottom_left] = new_matching[bottom_right] = OrderedDomino(
                    bottom_left, bottom_right)
            else:
                new_matching[top_right] = new_matching[bottom_right] = OrderedDomino(
                    top_right, bottom_right)    
        elif bottom_left in matching.keys() and matching[bottom_left].first == bottom_left \
        and matching[bottom_left].direction == 'horizontal':
            # Slide
            new_matching[v] = new_matching[top_right] = OrderedDomino(v, top_right)
        elif top_right in matching.keys() and matching[top_right].first == top_right \
        and matching[top_right].direction == 'vertical':
            # Slide
            new_matching[v] = new_matching[bottom_left] = OrderedDomino(v, bottom_left)
        else:
            # Create 2 dominos
            i = randint(0,1)
            if i == 0:
                new_matching[v] = new_matching[top_right] = OrderedDomino(v, top_right)
                new_matching[bottom_left] = new_matching[bottom_right] = OrderedDomino(bottom_left, bottom_right)
            else:
                new_matching[v] = new_matching[bottom_left] = OrderedDomino(v, bottom_left)
                new_matching[top_right] = new_matching[bottom_right] = OrderedDomino(top_right, bottom_right)
    return new_matching

In [ ]:
def make_cell_widget_class_index(matching, n):
    def cell_widget_class_index(pos):
        def calc_index_for_domino(d, n):
            if d.direction == 'horizontal':
                if not d.parity(n):
                    return 1
                else:
                    return 2
            else:
                if not d.parity(n):
                    return 3
                else:
                    return 4
        if pos in matching.keys():
            d = matching[pos]
            return calc_index_for_domino(d, n)  
        return 0
    return cell_widget_class_index

In [ ]:
%%html
<style>
.b1 {background-color: green}
.b2 {background-color: blue}
.b3 {background-color: red}
.b4 {background-color: yellow}
</style>

In [ ]:
from sage_combinat_widgets.grid_view_widget import GridViewWidget, BlankButton, styled_push_button
from ipywidgets import Layout
smallblyt = Layout(width='12px',height='12px', margin='0', padding='0')
Button0 = styled_push_button(disabled=True)
Button1 = styled_push_button(disabled=True, style_name='b1')
Button2 = styled_push_button(disabled=True, style_name='b2')
Button3 = styled_push_button(disabled=True, style_name='b3')
Button4 = styled_push_button(disabled=True, style_name='b4')

In [ ]:
ORDER = 12
g, m, md = figure(ORDER)
w = GridViewWidget(g, cell_layout=smallblyt, 
                   cell_widget_classes=[Button0, Button1, Button2, Button3, Button4], 
                   cell_widget_class_index=make_cell_widget_class_index(md, ORDER), 
                   blank_widget_class=BlankButton)
w

In [ ]:
for i in range(2,ORDER+1):
    md = tossing(g, ORDER, i, md)
    w.draw(cell_widget_class_index=make_cell_widget_class_index(md, i))